In [1]:
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import warnings
import time
import json

warnings.filterwarnings("ignore")


plt.style.use("fivethirtyeight")

sns.set_style({'font.sans-serif': ['simsun', 'Arial']})
sns.set_style('darkgrid', {'font.sans-serif': ['simhei', 'Arial']})
%matplotlib inline

holidays = pd.Series(json.load(open(r'../../Data/Holidays.json'))['holidays'])
subwayinfo = pd.read_csv(r'../../Data/BeijingSubway.csv')

In [2]:
df_copy = pd.read_csv(
    r'../../Data/data_set_phase1/train_queries.csv')
df = df_copy.copy()

# Datetime

In [4]:
df['req_time'] = pd.to_datetime(df['req_time'])
df['day_of_week'] = df['req_time'].dt.day_name()
df['req_date'] = df['req_time'].dt.strftime('%m-%d')
df['req_hour'] = df['req_time'].dt.hour
df['req_minute'] = df['req_time'].dt.minute
df['if_holiday'] = (df['req_date'].isin(holidays)).astype(int)

# Week day
current_c = list(df.columns.values)
weekdays = ['Monday', 'Tuesday', 'Wednesday',
            'Thursday', 'Friday', 'Saturday', 'Sunday']
current_c.extend(weekdays)
df = df.reindex(columns=current_c, fill_value=0)
for day_name in weekdays:
    df.loc[(df['day_of_week'] == day_name), [day_name]] = 1

# Hour
current_c = list(df.columns.values)
oclock = [str(i)+'_oclock' for i in range(0, 24)]
current_c.extend(oclock)
df = df.reindex(columns=current_c, fill_value=0)
for h in oclock:
    df.loc[(df['req_hour'] == int(h.split('_')[0])), [h]] = 1

In [ ]:
df.drop(columns=['day_of_week','req_date', 'req_hour'], inplace=True)

# Distance

In [ ]:
df['o_lng'] = df['o'].apply(lambda x: float(x.split(',')[0]))
df['o_lat'] = df['o'].apply(lambda x: float(x.split(',')[1]))
df['d_lng'] = df['d'].apply(lambda x: float(x.split(',')[0]))
df['d_lat'] = df['d'].apply(lambda x: float(x.split(',')[1]))
df['od_manhattan_distance'] = abs(
    df['o_lng']-df['d_lng'])+abs(df['o_lat']-df['d_lat'])

In [ ]:
df['o_nearest_dis'] = np.nan
df['d_nearest_dis'] = np.nan

In [ ]:
for index in df.index:
    df[index, 'o_nearest_dis'] = (abs(subwayinfo['station_longitude']-df.loc[index]
                                      ['o_lng'])+abs(subwayinfo['station_latitude']-df.loc[index]['o_lat'])).min()
    df[index, 'd_nearest_dis'] = (abs(subwayinfo['station_longitude']-df.loc[index]
                                      ['d_lng'])+abs(subwayinfo['station_latitude']-df.loc[index]['d_lat'])).min()

In [ ]:
df.head()